# UCI Adult Dataset Process Demo.
Only demonstrate the basic useage of this dataset and some algorithms in scikit library(And most copy with minimum changes from [【机器学习实验】scikit-learn的主要模块和基本使用](http://www.jianshu.com/p/1c6efdbce226)), if we want to have better and more meaningful results,we should do some more work.

## load the preprocessing: transform description to number([male,female]->[0,1])

In [1]:
def loadXandY(fileName):
    import numpy as np
    
    # load the CSV file as a numpy matrix
    with open(fileName, 'r') as f:
        dataset = np.loadtxt(f, delimiter=",")
    
    # separate the data from the target attributes
    attrsize = len(dataset[0])
    
    print(len(dataset), attrsize)
    
    X = dataset[:,0:-1]
    y = dataset[:,-1]
    return X,y

In [2]:
baseDir = 'H:/practice/scikit_class/scikit_learning/uci_adult/adult_data/'
# baseDir = 'adult_data/'

fileName = baseDir+'adult.data.num'

testFileName = baseDir+'adult.test.num'

X,y = loadXandY(fileName)
TX,Ty = loadXandY(testFileName)

32561 15
16281 15


# deal with the missing value
[Sci4.3.4. Encoding categorical features](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values)

In [3]:
def simpleImputer(X):
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(X)
    return imp.transform(X)

In [4]:
X = simpleImputer(X)
TX = simpleImputer(TX)

## The doc mentioned that we should use one hot encoding since `these expect continuous input, and would interpret the categories as being ordered, which is often not desired `.

See [scikit-learn机器学习RandomForest实例（含类别属性处理）](http://blog.csdn.net/mach_learn/article/details/40428297)
and [Note on using OneHotEncoder in scikit-learn to work on categorical features](https://xgdgsc.wordpress.com/2015/03/20/note-on-using-onehotencoder-in-scikit-learn-to-work-on-categorical-features/) for more info.

And our feature is:
![features](pics/describe.png)

In [5]:
def oneHotEncoderX(X):
    import numpy as np
    from sklearn import preprocessing
    to_change_features = list(map(lambda x:x-1, [2,4,6,7,8,9,10,14]))
#     enc = preprocessing.OneHotEncoder(categorical_features=to_change_features) # if not specify the n_values, by default, test set will be 104 long
    enc = preprocessing.OneHotEncoder(categorical_features=to_change_features, n_values=[8,16,7,14,6,5,2,41])
    enc.fit(X)
    return np.array(enc.transform(X).toarray())

In [6]:
X_oneHot = oneHotEncoderX(X)
TX_oneHot = oneHotEncoderX(TX)
print(X[1])
print(X_oneHot[1])
print(len(X_oneHot[1]))
print(len(TX_oneHot[1]))

[  5.00000000e+01   1.00000000e+00   8.33110000e+04   0.00000000e+00
   1.30000000e+01   0.00000000e+00   4.00000000e+00   2.00000000e+00
   0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   1.30000000e+01   0.00000000e+00]
[  0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+0

In [7]:
X = X_oneHot
TX = TX_oneHot

数据归一化(Data Normalization)

In [5]:
def simpleScale(X):
    from sklearn import preprocessing
    # normalize the data attributes
    normalized_X = preprocessing.normalize(X)
    # standardize the data attributes
    standardized_X = preprocessing.scale(X)
    return standardized_X

In [6]:
X = simpleScale(X)
TX = simpleScale(TX)

特征选择(Feature Selection)

In [7]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X, y)
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.17274419  0.04627868  0.16203893  0.03643286  0.08773375  0.0700939
  0.08065895  0.06733442  0.01465805  0.03654993  0.08149967  0.02940447
  0.09472681  0.01984539]


In [8]:
def show_predictions(y, predicted):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.scatter(y, predicted)
    ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()

In [9]:
def make_predictions(model, X, y):
    from sklearn import metrics
    # make predictions
    expected = y
    predicted = model.predict(X)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))
    print('\n')
#     show_predictions(expected[::len(expected)//20], predicted[::len(predicted)//20])

In [10]:
def cross_validation(model, X, y):
    from sklearn.model_selection import KFold, cross_val_score
    k_fold = KFold(n_splits=3)
    return cross_val_score(model, X, y, cv=k_fold, n_jobs=-1)

In [11]:
# 逻辑回归
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)
print(model)

# make_predictions(model, X, y)

make_predictions(model, TX, Ty)

# print(cross_validation(model, X, y))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

        0.0       0.72      0.54      0.62      3846
        1.0       0.87      0.94      0.90     12435

avg / total       0.83      0.84      0.83     16281

[[ 2094  1752]
 [  805 11630]]




In [12]:
# 朴素贝叶斯
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

GaussianNB(priors=None)
             precision    recall  f1-score   support

        0.0       0.69      0.40      0.51      7841
        1.0       0.83      0.94      0.88     24720

avg / total       0.80      0.81      0.79     32561

[[ 3160  4681]
 [ 1407 23313]]


             precision    recall  f1-score   support

        0.0       0.69      0.41      0.51      3846
        1.0       0.84      0.94      0.89     12435

avg / total       0.80      0.82      0.80     16281

[[ 1567  2279]
 [  707 11728]]


[ 0.8115902   0.81241938  0.81608772]


In [13]:
# K近邻
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
# fit a k-nearest neighbor model to the data
model = KNeighborsClassifier()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

        0.0       0.78      0.69      0.74      7841
        1.0       0.91      0.94      0.92     24720

avg / total       0.88      0.88      0.88     32561

[[ 5437  2404]
 [ 1504 23216]]


             precision    recall  f1-score   support

        0.0       0.66      0.58      0.62      3846
        1.0       0.88      0.91      0.89     12435

avg / total       0.83      0.83      0.83     16281

[[ 2241  1605]
 [ 1134 11301]]


[ 0.82623917  0.8291874   0.83331798]


In [ ]:
# 决策树
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
# fit a CART model to the data
model = DecisionTreeClassifier()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)


print(cross_validation(model, X, y))

In [ ]:
# This block is still about `决策树`

# The train set have only one outlier, so it's clearly overfit(since the test dataset is not so appealing),
# we want to have this outlier's info,since it can be interesting
import numpy as np
expected = y
predicted = model.predict(X)
diff = expected - predicted
diff_location = list(np.where(diff != 0)[0])
print(len(diff_location), diff_location)

def getLine(fileName, lineNum):
    with open(fileName, 'r') as f:
        all = f.readlines()
    
    return all[lineNum].strip()

getLine(baseDir+'adult.data', 848)

In [ ]:
# SVM
from sklearn import metrics
from sklearn.svm import SVC
# fit a SVM model to the data
model = SVC()
model.fit(X, y)
print(model)


make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

In [17]:
# 优化算法参数
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
# prepare a range of alpha values to test
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
# create and fit a ridge regression model, testing each alpha
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

H:\py_env\scikit_sys_site_packages\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
H:\py_env\scikit_sys_site_packages\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e+00,   1.00000e-01,   1.00000e-02,   1.00000e-03,
         1.00000e-04,   0.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.31229463325545104
1.0


In [18]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn.linear_model import Ridge
from sklearn.grid_search import RandomizedSearchCV
# prepare a uniform distribution to sample for the alpha parameter
param_grid = {'alpha': sp_rand()}
# create and fit a ridge regression model, testing random alpha values
model = Ridge()
rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100)
rsearch.fit(X, y)
print(rsearch)
# summarize the results of the random parameter search
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A213A70400>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)
0.31229463315387784
0.996920839648
